In [ ]:
!conda create -n intel -c intel intelpython3_full --yes

In [ ]:
!conda config --set ssl_verify false

In [ ]:
!conda install tensorflow-mkl --yes

In [ ]:
!conda install keras -c intel --yes

In [ ]:
!conda install jupyter -c intel --yes

In [ ]:
!conda init bash

In [ ]:
!activate intel

In [ ]:
!python

In [1]:
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [9]:
import os

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from dask.distributed import Client
from sklearn.externals import joblib
import pickle

/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [10]:
NUM_PARALLEL_EXEC_UNITS = 4
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=NUM_PARALLEL_EXEC_UNITS, inter_op_parallelism_threads=2,
                       allow_soft_placement=True, device_count={'CPU': NUM_PARALLEL_EXEC_UNITS})
session = tf.compat.v1.Session(config=config)
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["KMP_BLOCKTIME"] = "30"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"] = "granularity=fine,verbose,compact,1,0"

In [14]:
#Load and split the data
training_data = pd.read_csv('train.csv')
data = training_data.drop(['Target'], axis=1)
target = training_data['Target']

In [15]:
#Clean the data to either replace or remove string columns
data.select_dtypes(exclude=[np.number]).head()
data = data.select_dtypes(include=[np.number], exclude=[np.object]).fillna(0)

In [16]:
#Extract feature names
cols = data.columns.values
dicts = dict(zip(cols, range(len(cols))))

features = []
for key in dicts:
    if dicts[key] in [98, 135, 134, 131, 118, 133, 109, 132, 94, 2]:
        features.append(key)

In [17]:
#Define train and test data sets
X_train, X_test, y_train, y_test = train_test_split(data[features], target, test_size= 0.1, random_state=42)
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [18]:
%timeit
model = tf.keras.Sequential()
model.add(Dense(8, input_dim=10, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(6, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs = 500, batch_size = 132)
y_pred = model.evaluate(X_test, y_test)
print("Test Accuracy: " + str(round(y_pred[1]*100,2)) + '%')

Train on 8601 samples
Epoch 1/500
8601/8601 [==============================] - 1s 92us/sample - loss: 2.9752 - accuracy: 0.5942
Epoch 2/500
8601/8601 [==============================] - 0s 19us/sample - loss: 1.9919 - accuracy: 0.5704
Epoch 3/500
8601/8601 [==============================] - 0s 18us/sample - loss: 1.4246 - accuracy: 0.5235
Epoch 4/500
8601/8601 [==============================] - 0s 19us/sample - loss: 1.1794 - accuracy: 0.5760
Epoch 5/500
8601/8601 [==============================] - 0s 19us/sample - loss: 1.0586 - accuracy: 0.6248
Epoch 6/500
8601/8601 [==============================] - 0s 19us/sample - loss: 1.0185 - accuracy: 0.6298
Epoch 7/500
8601/8601 [==============================] - 0s 19us/sample - loss: 1.0002 - accuracy: 0.6331
Epoch 8/500
8601/8601 [==============================] - 0s 18us/sample - loss: 0.9870 - accuracy: 0.6343
Epoch 9/500
8601/8601 [==============================] - 0s 19us/sample - loss: 0.9785 - accuracy: 0.6346
Epoch 10/500
8601/8601 [